Requêtage restaurant

Requete API

In [50]:
import requests
from sqlalchemy import create_engine, text
import pandas as pd
from SQL_user import SQL_user


with open('apikeyrm.txt', 'r') as file: #mettre clé api
    API_KEY = file.read().strip()

QUERY = 'cafeteria toulouse gramont'


# URL de l'API Places

url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={QUERY}&key={API_KEY}&sort_by=rating"
# Envoyer la requête

response = requests.get(url)
# data = response.json()
data2 = response.json()

In [48]:
sql_user= SQL_user()


# Chaîne de connexion
engine = sql_user.engine

In [51]:


nb_result = 20

# Utilisation de compréhension de dictionnaire pour créer le dictionnaire de la query
dict_adresses = {
    'restaurant_id': [result['name'] + str(result['geometry']['location']['lat']) for result in data2['results'][:nb_result]],
    'name' : [result['name'] for result in data2['results'][:nb_result]],
    'formatted_address': [result['formatted_address'] for result in data2['results'][:nb_result]],
    'lat': [result['geometry']['location']['lat'] for result in data2['results'][:nb_result]],
    'lng' : [result['geometry']['location']['lng'] for result in data2['results'][:nb_result]],
    'photo_reference' : [result['photos'][0]['photo_reference'] for result in data2['results'][:nb_result]],
    'rating': [result['rating'] for result in data2['results'][:nb_result]],
    'user_ratings_total' : [result['user_ratings_total'] for result in data2['results'][:nb_result]],
}

#creation dataframe de la query:

df = pd.DataFrame(dict_adresses)
#connection à postgrey SQL à modifier avec create_engine(url)
engine = sql_user.engine

#Convertir la colonne "name" en liste
id_list = df["restaurant_id"].tolist()


#Générer les paramètres dynamiquement (:name0, :name1, etc.)
param_dict = {f"restaurant_id{i}": id for i, id in enumerate(id_list)}
placeholders = ", ".join(f":restaurant_id{i}" for i in range(len(id_list)))

#Requête SQL avec IN
query = text(f"SELECT * FROM restaurants WHERE restaurant_id IN ({placeholders})")

#Exécution de la requête
with engine.connect() as connection:
    result = connection.execute(query, param_dict)
    restaurants = result.fetchall()

restaurants = pd.DataFrame(restaurants)
display(restaurants)
if len(restaurants) !=0 :
    restaurants = set(restaurants["restaurant_id"].unique())  # Convertir en ensemble d'IDs
df_push = df.loc[~df["restaurant_id"].isin(restaurants)]
# Filtrer correctement

df_push.to_sql(name='restaurants', con=engine, if_exists = 'append', index=False)

""


20

verification et identification client

In [10]:
#entrée utilisateur

email = input('e-mail')

secret_pass = input('Password')



In [103]:
#verification idenfication


identification = 0

while identification == 0:

    email = input('e-mail')
    secret_pass = input('Password')
    query = text("SELECT * FROM user WHERE mail = :email AND password = :secret_pass")
        #Exécution de la requête
    with engine.connect() as connection:
        result = connection.execute(query, {'email': email, 'secret_pass': secret_pass})
        identification = result.fetchall()
    identification = pd.DataFrame(identification)
    display(identification)
    if len(identification) == 0 :
        display('Mail ou mot de passe incorrect')
        identification = 0
    else :
        identification = len(identification)

display('affichage page principale')

    

""


'Mail ou mot de passe incorrect'

,id,nom,prenom,mail,password
0,1,Martin,Jean,jean.martin@test.com,@test.com


'affichage page principale'

In [139]:
import re

new_user_mail = input('e-mail')
new_user_name = input('Name')
new_user_lastname = input('Lastname')
new_user_secret_pass = input('Password')

def checking(new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass):
    #regex de qui vont servir de verification
    regex_mail = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    regex_password = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'
    query = text("SELECT * FROM user WHERE mail = :new_user_mail")
    with engine.connect() as connection:
            result = connection.execute(query, {'new_user_mail': new_user_mail})
            creation = result.fetchall()
    creation = pd.DataFrame(creation)
    if len(creation) == 0:
        creation.to_sql(name='user', con=engine, if_exists = 'append', index=False)
    else :
        return "email déjà connu merci de vous identifier"  
    if re.match(regex_mail, new_user_mail):
        new_user_mail = True
        if len(new_user_name) !=0:
            new_user_name = True
            if len(new_user_lastname) !=0:
                new_user_lastname = True
                if re.match(regex_password, new_user_secret_pass):
                    new_user_secret_pass = True
                else:
                    new_user_secret_pass = False
            else:
                new_user_lastname = False
        else:
            new_user_name = False  
    else:
        new_user_mail = False
    chek_in = [new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass]
    return chek_in

checking(new_user_mail, new_user_name, new_user_lastname, new_user_secret_pass)



[True, True, True, True]

In [11]:
import re

# Exemple de mot de passe à tester
password = '"rèsT"658!'

# Expression régulière pour valider un mot de passe
regex = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'

# Vérification du mot de passe
if re.match(regex, password):
    print("Mot de passe valide")
else:
    print("Mot de passe invalide")


Mot de passe valide


In [26]:
class Checker:
    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    password_regex = r'^(?=.*[a-zàâäéèêëîïôöùûüÿçáéíñóúü])(?=.*[A-ZÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ])(?=.*\d)(?=.*[@$!%*?&_\"\'\'])[A-Za-zàâäéèêëîïôöùûüÿçáéíñóúüÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇÁÉÍÑÓÚÜ\d@$!%*?&_\"\'\']{8,}$'

    def __init__(self, engine):
        self.engine = engine

    def is_email_known(self, email):
        query = text("SELECT COUNT(*) FROM user WHERE email = :email")
        with self.engine.connect() as connection:
            result = connection.execute(query, {'email': email})
            count = result.scalar()
        return count > 0

    @staticmethod
    def is_valid_email(email):
        return re.match(Checker.email_regex, email) is not None

    @staticmethod
    def is_valid_password(password):
        return re.match(Checker.password_regex, password) is not None

    @staticmethod
    def is_valid_name(name):
        return bool(name) and all(char.isalpha() or char.isspace() for char in name)

    @staticmethod
    def is_valid_lastname(lastname):
        return bool(lastname) and all(char.isalpha() or char.isspace() for char in lastname)

    def validate_user(self, email, password, name, lastname):
        if not self.is_valid_email(email):
            print("Mail invalide")
            return False
        if not self.is_valid_password(password):
            print("Mot de passe invalide merci de renseigner minimum 8 caractères dont un chiffre, majuscule et symbole")
            return False
        if not self.is_valid_name(name):
            print("Name invalid")
            return 
        if not self.is_valid_lastname(lastname):
            print("Lastname invalid")
            return 
        return True
    # alternatif
    def val(self, email, password, name, lastname):
        return not self.is_valid_email(email) and not self.is_valid_password(password) and not self.is_valid_name(name) and not self.is_valid_lastname(lastname)

    @staticmethod
    def add_user(email, password, name, lastname):
        dict_user = {'nom': [name], 'prenom': [lastname], 'mail': [email], 'password': [password]}
        df_user = pd.DataFrame(dict_user)
        df_user.to_sql(name='user', con=engine, if_exists = 'append', index=False)
    
    
    def is_authentificate(self, email, password):
        query = text("SELECT COUNT(*) FROM user WHERE mail = :email and password = :password")
        with self.engine.connect() as connection:
            result = connection.execute(query, {'email': email, 'password': password})
            count = result.scalar()
        return count > 0

# Exemple d'utilisation
email = "r.meson@test.com"
password = "WCS_Rémi_2025"
name = "Jean"
lastname = "Dupont"



In [2]:
# Exemple d'utilisation
email = "r.meson@test.com"
password = "WCS_Rémi_2025"
name = "Jean"
lastname = "Dupont"



In [3]:
from checker import Checker

checker= Checker()

checker.is_email_known(email)

False

In [35]:


if checker.is_email_known(email):
    print("Mail déjà connu merci de vous identifier")
else:
    if checker.validate_user(email, password, name, lastname):
        checker.add_user(email, password, name, lastname)

Mail déjà connu merci de vous identifier


In [3]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------------------------------------ --- 2.4/2.6 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 11.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
